# Lecture 6: simulating quantum noise

<center><img src="../../figures/qibo_mascotte/qibo_noise.png" alt="drawing" width="400"/></center>
<center><strong>Fig. 6:</strong> Qibo the mangoose suffering noise [DALL-E].</center>

## Introduction

In real life scenarios, quantum computers are imperfect and prone to noise. Some of the noise sources can be:
- **decoerence**, which corresponds to the loss of the quantum properties of the system, usually due to some interaction with the external environment;
- **thermal noise**, which affects in particular systems sensitive to temperature;
- **control errors**, due to some imperfections in the laboratory tools (cables, control boards, etc.);
- **cross-talk**, which happens when some operation we apply to a single qubit unintentionally affect the neighbouring qubits;
- **quantum state leakage**, which is the possibility to access some undesired energy level. It can happen for example using superconducting qubits, which are in principle N-levels systems prepared to isolate the first two energy levels.

More in general, a combination of these phenomena can affect a real quantum device, leading to a simple but relevant problem: the expected results are corrupted. 

In `qibo`, we can simulate noisy systems and implement some strategies to take this problem into account, cleaning our results from undesired corruptions. 

In the following:

1. we define a simple problem, consisting in calculating the expected value of a target observable over the final state we obtain executing a quantum circuit;
2. we compute the exact result with noiseless simulation;
3. we add noise to the circuit and compute the noisy result;

#### 1. Problem definition

We consider as an example a simple quantum system of three qubits, on which we aim to apply some rotations and then compute the expectation value of an observable composed of Pauli's.

In [1]:
# import numpy and matplotlib

# import qibo and its modules (Circuit, gates, hamiltonians)

# import plotscripts from qiboedu.scripts

In [2]:
# set qibo's backend

In [4]:
# define a parametrized circuit
nqubits = 3
nlayers = 3

c = Circuit(nqubits)
for l in range(nlayers):
    for q in range(nqubits):
        c.add(gates.RY(q=q, theta=0))
        c.add(gates.RZ(q=q, theta=0))
    for q in range(0, nqubits-1, 1):
        c.add(gates.CNOT(q0=q, q1=q+1))
c.add(gates.M(*range(nqubits)))

MeasurementResult(qubits=(0, 1, 2), nshots=0)

In [3]:
# draw the circuit

Now the circuit is set up and we can fix its action by injecting a specific list of rotational angles. 

We get the numbers of contained parameters:

In [4]:
# save total number of circuit's parameters

And then we generate a well defined set of angles.

In [7]:
# new angles
angles = np.linspace(0, 2*np.pi, nparams)
print(angles)

[0.         0.36959914 0.73919827 1.10879741 1.47839654 1.84799568
 2.21759481 2.58719395 2.95679309 3.32639222 3.69599136 4.06559049
 4.43518963 4.80478876 5.1743879  5.54398704 5.91358617 6.28318531]


In [5]:
# set the angles into the circuit

Now we have a quantum circuit composed of rotations and some CNOT gates. We want to use it to compute some calculations. 

To do this, we can define an observable $\mathcal{O}$, for which we calculate:

$$ \langle \mathcal{O} \rangle \equiv \langle 0 | U^{\dagger}\, \mathcal{O}\, U | 0 \rangle \\. $$

We make the choice:

$$ \mathcal{O} = - \sum_{i=0}^N Z_i \\.$$

In [6]:
# observable definition
# global Z using qibo's hamiltonians

#### 2. Computing the exact expectation value

In [7]:
# execute the circuit and save outcome object

# print final state

# collect and visualize frequencies

In [8]:
# compute the expected value

#### 3. Add noise to the system

Various different ways to simulate noise exist in `qibo`, but in this lecture we focus on one of them. 

Let me introduce before the Pauli noise channel, which is used here to corrupt the circuit. If we consider a quantum state represented by the density matrix $\rho$, the effect of a Pauli noise channel on the state is

$$ \mathcal{E}(\rho) = \biggl(1 - \sum_{k=0}p_k\biggr)\rho + \sum_k P_k \,\rho\, P_k \\,
$$

where $P_k$ is the $k$-th Pauli string and $p_k$ is the probability of applying $P_k$. 

This channel allows to formalize a corruption of the state $\rho$ in which each Pauli represents some specific error which can occurr: for example, the Pauli X and Z respectively represent a bit-flip error and a phase-flip error in the qubit state.

We are going to use the `circuit.with_pauli_noise` method, which consists in applying a Pauli noise channel after every gate of the circuit. 

In [9]:
# define the Pauli's probabilities for each qubit of the system
noise_map = {
    0: list(zip(["X", "Z"], [0.007, 0.008])),
    1: list(zip(["Y", "Z"], [0.01, 0.005])),
    2: list(zip(["X", "Z"], [0.008, 0.007]))
}

# use circuit.with_pauli_noise method

Let's print the new face of the circuit, with the Pauli noise channel after each gate.

In [10]:
# draw noisy circuit

In [11]:
# set the previously defined parameters

In [12]:
# execute the circuit

# collect and visualize frequencies
# you can pass as argument two frequencies objects

As final step of this lecture 4, let's compute the expectation value of $\mathcal{O}$ over the noisy final state.

In [13]:
# compute noisy expectation value

# compare it with the noiseless circuit

<div style="background-color: rgba(255, 105, 105, 0.3); border: 2.5px solid #000000; padding: 15px;">
    <strong>Exercise:</strong> what is happening to the system? why is the expected value of $Z$ decreasing? What happens if you increase the magnitude of the noise?
</div>